- ## LDA (Latent Dirichlet Allocation) для разделение текстов новостей, описаний опасных природных явлений, ущербов, мер и мероприятий по тематикам.
- ## `Random Forest` как модель предсказания интенсивности ветра по тексту новости

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import pickle
from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doс)
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation

### Загружаем данные с ветрами

In [3]:
wind = pd.read_excel('files//wind_bof_dis.xlsx')

In [4]:
wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12734 entries, 0 to 12733
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12734 non-null  int64         
 1   _ID                     12734 non-null  object        
 2   Event_ID                11005 non-null  object        
 3   News_link               12734 non-null  object        
 4   Text                    12734 non-null  object        
 5   News_length             12734 non-null  int64         
 6   Количество              12734 non-null  int64         
 7   News_source             12734 non-null  object        
 8   News_type               12734 non-null  object        
 9   Location_ATD4           12679 non-null  object        
 10  Location_other          11201 non-null  object        
 11  Location_geocoded       12734 non-null  object        
 12  Location_geocoded_bool  12734 non-null  bool  

In [21]:
wind['Класс'].value_counts()

Класс
0     4272
8     2582
10    1913
9     1826
7     1036
11     447
12     281
6      181
5      152
4       29
3       14
2        1
Name: count, dtype: int64

In [98]:
print(f'Длина датасета с ветрами: {len(wind)}')
print(f'Длина части датасета с ветрами с ' +
      f'определенным классом по Бофорту: {len(wind[wind['Класс']>0])}, ' +
      f'что составляет - {len(wind[wind['Класс']>0])/len(wind)*100:.2f} %')
print(f'Длина части датасета с ветрами, ' +
      f'рассматриваемого для обучения: {len(wind[wind['Класс']>4])}, ' +
     f'что составляет - {len(wind[wind['Класс']>4].dropna(subset = 'Text'))/len(wind)*100:.2f} %')

Длина датасета с ветрами: 12734
Длина части датасета с ветрами с определенным классом по Бофорту: 8462, что составляет - 66.45 %
Длина части датасета с ветрами, рассматриваемого для обучения: 8418, что составляет - 66.11 %


In [182]:
# Процент новостей про сильные ветры, где указана скорость ветра

round(wind[wind['Класс']>0]['News_source'].value_counts()/wind['News_source'].value_counts()*100, 2)

News_source
ВКонтакте    70.67
МЧС          42.61
СМИ          39.22
Name: count, dtype: float64

### Делаем 4 датафрейма для дальнейшей работы

In [5]:
wind_work = wind[['Text', 'News_type', 'Location_ATD4', 'Phenomena', 'Класс']]
event = wind[['Event_verbose', 'News_type', 'Location_ATD4', 'Phenomena', 'Класс']]
damage = wind[['Damage_description', 'News_type', 'Location_ATD4', 'Phenomena', 'Класс']]
activities = wind[['Activities', 'News_type', 'Location_ATD4', 'Phenomena', 'Класс']]

### Обычная токенизация на примере `wind_work`

In [6]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mariia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Mariia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mariia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
def normalise_text(text):
    text = text.lower()

    # сурово регулярками выкидываем мусорные символы
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    return text.strip()

wind_work['text_clean'] = wind_work.Text.apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(wind_work.text_clean.values)))
len(word_cnt)

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\386940470.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_work['text_clean'] = wind_work.Text.apply(normalise_text)


57412

In [8]:
from nltk.corpus import stopwords

stops_ru = set(stopwords.words('russian'))
len(stops_ru)

151

In [9]:
word_cnt.most_common()[:10]

[('в', 67864),
 ('и', 67006),
 ('на', 38311),
 ('по', 32792),
 ('с', 25130),
 ('не', 16919),
 ('при', 14600),
 ('до', 13374),
 ('от', 11019),
 ('мс', 10405)]

In [10]:
wind_voc = {
    "#PAD#": 0, "#UNK#": 1
}

for word, _ in word_cnt.most_common():
    if word not in stops_ru:
        wind_voc[word] = _

In [11]:
# то же самое через функцию
def create_vocab(text, stops_ru=stops_ru):

    word_cnt = Counter(word_tokenize(text))
    vocabulary = {
        "#PAD#": 0, "#UNK#": 1
    }

    k = 2
    for word, _ in word_cnt.most_common():
        if word not in stops_ru:
            vocabulary[word] = k
            k += 1
    return vocabulary

wind_voc = create_vocab(' '.join(wind_work.text_clean.values))

### Сделаем токенизацию с помощью библиотеки Наташа: обрабатываем текст, выделяя ключевые элементы и их контекст

In [17]:
# Инициализация
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def process_text(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    # Лемматизация и фильтрация токенов
    processed_tokens = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        # Пропускаем знаки препинания и служебные части речи
        if token.pos not in ('PUNCT', 'CCONJ', 'SCONJ', 'ADP', 'PART'):
            processed_tokens.append(token.lemma)
    
    return processed_tokens
def extract_context(tokens, index, window=4):
    start = max(0, index - window)
    end = min(len(tokens), index + window + 1)
    return tokens[start:end]

In [16]:
total = 0
for i in wind_work['Text']:
    print()
    tokens = process_text(i)  

    if tokens:
        context = extract_context(tokens, index=0)
        print(context)
    
    total += 1
    if total > 10: 
        break



['ангарский', 'округ', 'устранять', 'последствие', 'непогода']

['пострадать', 'чс', 'приморец', 'мочь', 'подать']

['центральный', 'россия', 'надвигаться', '9-балльный', 'шторм']

['рязанцев', 'предупреждать', 'ухудшение', 'погодный', 'условие']

['югра', 'березовскийрайон', 'лес', 'югра', 'завершиться']

['внимание', 'экстренный', 'предупреждение', 'данные', 'метеорологический']

['администрация', 'район', 'помочь', 'заменить', 'текущий']

['рязань', 'близкий', 'время', 'ожидаться', 'гроза']

['14', 'июль', 'поселок', 'болонь', 'ураган']

['евпатория', 'высадить', 'более', '2700', 'дерево']

['ожидаться', 'ухудшение', 'погодный', 'условие', 'связь']


### Сделаем LDA для
- Текста
- Описания ОПЯ
- Описания ущерба
- Мер и мероприятий

Распределим по классам интенсивности ОПЯ

In [19]:
def to_lda(df, tokens='tokens', ngram_range=(1, 1), n_themes = 5, n_words = 15):
    res_dict = {5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}}
    
    # Список токенов для исключения
    custom_stop_words = ['м', 'с', 'мм', 'c', 'км', 'метр',
                         'секунда', 'лицензия', 'creative', 'commons', 'attribution']
    
    # Настройка CountVectorizer для игнорирования чисел и указанных токенов
    vectorizer = CountVectorizer(
        token_pattern=r'\b[a-zA-Zа-яА-ЯёЁ]+\b', 
        stop_words=custom_stop_words,
        ngram_range=ngram_range,
        max_features=1000
    )
    
    for cl in range(5, 13):
        sample = df[df['Класс'] == cl]
        
        # Проверка на наличие данных
        if len(sample) == 0:
            continue
        
        # Векторизация токенов
        X = vectorizer.fit_transform(sample[tokens])
        
        # Инициализация и обучение LDA
        lda = LatentDirichletAllocation(n_components=n_themes, random_state=42)
        lda.fit(X)
        
        words = vectorizer.get_feature_names_out()
        
        # Извлечение матрицы тем и слов
        topic_word_matrix = lda.components_
        
        # Количество слов, которые вы хотите отобразить для каждой темы
        num_top_words = n_words
        
        # Вывод результатов
        for topic_idx, topic in enumerate(topic_word_matrix):
            # Получаем индексы слов, отсортированных по вероятности
            top_word_indices = topic.argsort()[-num_top_words:][::-1]
            top_words = [words[i] for i in top_word_indices]
            t = f"Тема {topic_idx + 1}:"
            res_dict[cl][t] = top_words
            
    return res_dict


In [20]:
wind_work['tokens'] = wind_work['Text'].apply(lambda x: ' '.join(process_text(x)))
wind_dict = to_lda(wind_work,'tokens', ngram_range=(3, 8), n_themes = 8, n_words = 15)

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\596086053.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_work['tokens'] = wind_work['Text'].apply(lambda x: ' '.join(process_text(x)))


In [190]:
wind_dict_df = pd.DataFrame(wind_dict)
wind_dict_df

,5,6,7,8,9,10,11,12
Тема 1:,"[переменный облачность ночь место, переменный ...","[температура воздух ночь, воздух ночь день, те...","[гу мчс россия, лоджий предмет который, которы...","[мчс россия липецкий, мчс россия липецкий обла...",[комплекс неблагоприятный метеорологический яв...,"[шквалистый усиление ветер, очень сильный дожд...","[место очень сильный, предел населенный пункт,...","[риа новость дмитрий, риа новость дмитрий маке..."
Тема 2:,"[отдельный участок автодорога, республика башк...","[обильный снегопад метель, метель порывистый в...","[застать вы улица, сильный ветер застать вы ул...","[мониторинг окружать среда, гидрометеорология ...","[единый номер вызов, номер вызов экстренный, е...",[среднесибирский управление гидрометеорология ...,"[ожидаться очень сильный, застать вы улица, ве...","[материал сайт доступный, весь материал сайт д..."
Тема 3:,"[температура воздух ночь, гу мчс россия, мчс р...","[мониторинг окружать среда, гидрометеорология ...","[весь материал сайт доступный, весь материал с...","[возникновение чрезвычайный ситуация, налипани...","[стоять линия электропередача, линия электропе...","[остановка общественный транспорт, ветер заста...","[возникновение чрезвычайный ситуация, управлен...","[гидрометеорология мониторинг окружать, гидром..."
Тема 4:,"[мчс россия пермский край, мчс россия пермский...","[управление мчс россия, главный управление мчс...","[мониторинг окружать среда, гидрометеорология ...","[мчс россия республика коми, россия республика...","[шквалистый усиление ветер, мониторинг окружат...","[управление мчс россия, главный управление мчс...","[провод линия электропередача, оборванный пров...","[беречь себя свой, себя свой близкие, беречь с..."
Тема 5:,"[гололедно изморозевый отложение, облачность п...","[мчс россия пермский край, россия пермский кра...","[усиление ветер порыв, быть внимательный остор...","[застать вы улица, стена дом так, прятаться си...","[мчс россия республика коми, россия республика...","[сильный дождь ливень, гроза град шквалистый, ...","[подъем уровень вода, очень сильный дождь, риа...","[усиление ветер место, залив петр великий, гла..."
Тема 6:,"[г о г, гидрометеорология мониторинг окружать,...","[гидрологический явление прогнозироваться, явл...","[мчс россия липецкий область, мчс россия липец...","[град шквалистый усиление, град шквалистый уси...","[найти укрытие дерево, пытаться найти укрытие ...","[возникновение чрезвычайный ситуация, воздержа...",[стационарный адрес кабардино балкарский респу...,"[быть предельно осторожный, порыв ветер достиг..."
Тема 7:,"[температура воздух ночь, главный управление м...","[снег отдельный район, гу мчс россия, мчс росс...","[гу мчс россия, доверие гу мчс, телефон довери...","[весь материал сайт, весь материал сайт доступ...","[сильный дождь ливень, ливень сочетание гроза,...","[правило дорожный движение, соблюдать правило ...","[ухудшение погодный условие, сильный ветер пор...","[предел населенный пункт, северо западный вете..."
Тема 8:,"[небольшой снег ветер, возникновение чрезвычай...","[мчс россия пермский, россия пермский край, мч...","[слабо укрепить конструкция, центр гидрометеор...","[шквалистый усиление ветер, гроза шквалистый у...","[дождь гроза град, ожидаться сильный дождь, до...","[ожидаться очень сильный, возникновение чрезвы...","[весь материал сайт, весь материал сайт доступ...","[аварийный служба устранять, служба устранять ..."


In [25]:
def to_clean(value):
    return str(value)

In [28]:
event = event.dropna().reset_index(drop = True)
event['Event_verbose'] = event['Event_verbose'].apply(to_clean)
event['tokens'] = event['Event_verbose'].apply(lambda x: ' '.join(process_text(x)))
event_dict = to_lda(event, ngram_range=(3, 5), n_themes = 8, n_words = 10)

In [191]:
event_dict_df = pd.DataFrame(event_dict)
event_dict_df

,5,6,7,8,9,10,11,12
Тема 1:,"[отдельный участок автодорога, участок автодор...","[гололедно изморозевый отложение, умеренный сн...","[гололедица снежный занос, сильный дождь ливен...","[юго западный ветер, западный ветер порыв, тем...","[сильный дождь гроза, сильный дождь гроза град...","[сочетание гроза град, ливень сочетание гроза,...","[северо восточный ветер, северо западный ветер...","[место очень сильный, усиление ветер место, пе..."
Тема 2:,"[температура ночь прояснение, температура ночь...","[место сильный снег, снег место сильный снег, ...","[усиление ветер порыв, ливневый дождь гроза, о...","[усиление ветер порыв, ожидаться сильный дождь...","[шквалистый усиление ветер, гроза шквалистый у...","[восточный ветер порыв, сильный снег метель, д...","[порыв ветер достигать, ожидаться очень сильны...","[место очень сильный, усиление ветер место, ож..."
Тема 3:,"[снег мокрый снег, север восток место, осадок ...","[температура ночь день, ветер юго восточный, а...","[налипание мокрый снег, мокрый снег провод, до...","[налипание мокрый снег, мокрый снег провод, сн...","[ливень сочетание гроза, град шквалистый усиле...","[очень сильный ветер, сильный ветер порыв, оче...","[очень сильный ветер, место очень сильный, вид...","[дождь мокрый снег, очень сильный дождь, дождь..."
Тема 4:,"[место туман видимость, ночь место день, темпе...","[температура воздух ночь, гололедно изморозевы...","[шквалистый усиление ветер, юго западный ветер...","[порыв температура ночь, дождь мокрый снег, сн...","[гроза град шквалистый, шквалистый усиление ве...","[сильный дождь ливень, шквалистый усиление вет...","[сильный ветер порыв, сильный дождь ветер, ожи...","[восточный юго восточный, гололедица усиление ..."
Тема 5:,"[температура воздух ночь, ниже климатический н...","[температура воздух ночь, ветер юго восточный,...","[северо восточный ветер, снег место сильный, с...","[сильный дождь ливень, шквалистый усиление вет...","[подъем уровень вода, превышение неблагоприятн...","[ливень крупный град, сильный ливень крупный г...","[сильный осадок смешанный, сильный осадок смеш...","[усиление ветер место, подъем уровень море, ве..."
Тема 6:,"[температура воздух ночь, ниже климатический н...","[аномально холодный погода, воздух ночь день, ...","[температура ночь день, порыв температура ночь...","[шквалистый усиление ветер, ожидаться сильный ...","[усиление ветер порыв, шквалистый усиление вет...","[очень сильный дождь, сильный дождь сильный, с...","[ветер место порыв, усиление ветер место порыв...","[очень сильный осадок, дождь переходить снег, ..."
Тема 7:,"[ночь прояснение день, ветер южный юго, темпер...","[температура воздух ночь, снежный занос ветер,...","[снег мокрый снег, мокрый снег дождь, сильный ...","[ливень сочетание гроза, сочетание гроза град,...","[очень сильный дождь, сильный ветер порыв, сил...","[шквалистый усиление ветер, очень сильный дожд...","[сильный ветер порыв, усиление ветер гора, ожи...","[метель видимость менее, очень сильный ветер, ..."
Тема 8:,"[ночь день ветер, температура воздух ночь, ноч...","[место умеренный снег, небольшой место умеренн...","[температура воздух ночь, юго восточный ветер,...","[гроза шквалистый усиление, шквалистый усилени...","[град усиление ветер, северо западный ветер, п...","[северо западный ветер, усиление северо западн...","[юго восточный ветер, очень сильный юго восточ...","[ветер переход северо, дождь место очень, дожд..."


In [30]:
damage = damage.dropna().reset_index(drop = True)
damage['Damage_description'] = damage['Damage_description'].apply(to_clean)
damage['tokens'] = damage['Damage_description'].apply(lambda x: ' '.join(process_text(x)))
damage_dict = to_lda(damage, ngram_range=(3, 5), n_themes = 8, n_words = 10)

In [193]:
damage_dict_df = pd.DataFrame(damage_dict)
damage_dict_df

,5,6,7,8,9,10,11,12
Тема 1:,"[трудность безопасный управление, трудность бе...","[г тимашевск ул, шифер г тимашевск ул, шифер г...","[дорожно транспортный происшествие, гололедица...","[слабо укрепить конструкция, обрушение слабо у...","[отключение трансформаторный подстанция, транс...","[обрыв линия электропередача, слабо укрепить к...","[падение слабозакрепленный конструкция, возмож...","[обрыв линия электропередача, выход река берег..."
Тема 2:,"[ухудшать видимость дорога создавать больший, ...","[двухквартирный дом один, двухквартирный дом о...","[лэп линия связь, увеличение количество дтп, в...","[весь вид транспорт, работа весь вид, работа в...","[повреждение кровля здание, падение дерево обр...","[обрыв линия электропередача, обрушение слабо ...","[падение дерево слабо, филиал россеть центр пр...","[авария объект энергетика, авария объект энерг..."
Тема 3:,"[отдельный участок дорога, задеть автомобиль т...","[убирать улично дорожный сеть, убирать улично ...","[неблагоприятный погодный условие, дорога снеж...","[возникновение чрезвычайный ситуация, вероятно...","[обрыв линия связь, обрыв линия связь электроп...","[обрыв линия электропередача, авария объект эн...","[нарушение работа система, нарушение работа си...","[владивосток повредить кровля несколько дом, в..."
Тема 4:,"[обрыв линия электропередача, дорожно транспор...","[привести образование снежный, дорога рекоменд...","[возникновение чрезвычайный ситуация, возникно...","[повреждение кровля здание, падение дерево обр...","[обрыв линия электропередача, линия электропер...","[слабо закрепить конструкция, возникновение чр...","[снежный накат занос, снежный накат занос отде...","[высока вероятность возникновение, дренажно ко..."
Тема 5:,"[дорога гололедица снежный, вероятность занос ...","[возможный образование гололедица, дорога возм...","[линия связь электропередача, обрыв линия связ...","[дорожно транспортный происшествие, обрыв лини...","[возникновение происшествие связать, нарушение...","[нарушение система жизнеобеспечение, система ж...","[погодный условие мочь, неблагоприятный погодн...","[нарушение работа система, нарушение работа тр..."
Тема 6:,"[трудность безопасный управление, трудность бе...","[соблюдать скоростной режим, необходимый быть ...","[рекламный щит падение, обрыв линия электропер...","[нарушение система жизнеобеспечение, линия свя...","[весь вид транспорт, линия связь электропереда...","[федеральный региональный трасса, движение мар...","[режим повысить готовность, результат сильный ...","[увеличение количество дорожно, увеличение кол..."
Тема 7:,"[кравчук подписать постановление, краевой цент...","[обрыв линия электропередача, налипание мокрый...","[дорога возможный гололедица, возможный гололе...","[линия связь электропередача, обрыв линия связ...","[возникновение чрезвычайный ситуация, чрезвыча...","[кровля остекление здание, дерево обрыв провод...","[подъем уровень вода, приостановить автобусный...",[нарушение транспортный сообщение затруднение ...
Тема 8:,"[что мочь привести, вероятность занос автомоби...",[недопустимо торможение одновременный поворот ...,"[ухудшать видимость дорога, выбить стекло пада...","[увеличение количество дтп, лэп линия связь, в...","[электроснабжение населить пункт, нарушение эл...","[затруднение работа автотранспорт, воздушный л...","[специалист единица спецтехника, ликвидация те...","[рекламный конструкция повреждение кровля, эле..."


In [32]:
activities = activities.dropna().reset_index(drop = True)
activities['Activities'] = activities['Activities'].apply(to_clean)
activities['tokens'] = activities['Activities'].apply(lambda x: ' '.join(process_text(x)))
activities_dict = to_lda(activities, ngram_range=(3, 5), n_themes = 8, n_words = 10)

In [192]:
activities_dict_df = pd.DataFrame(activities_dict)
activities_dict_df

,5,6,7,8,9,10,11,12
Тема 1:,"[соблюдать скоростной режим, выбирать скоростн...","[мчс россия курский область, мчс россия курски...","[соблюдать скоростной режим, скоростной режим ...","[режим повысить готовность, возникновение чрез...","[рекомендовать население воздержаться, дерево ...","[режим повысить готовность, ограничить выход з...","[воздержаться поездка предел, главный управлен...","[предел населенный пункт, уточнить местонахожд..."
Тема 2:,"[главный управление мчс, управление мчс россия...","[главный управление мчс, главный управление мч...","[поездка личный автотранспорт, безопасный скор...","[ухудшение погодный условие, мера личный безоп...","[ограничить выход здание, мера личный безопасн...","[быть внимательный осторожный, соблюдать мера ...","[возникновение чрезвычайный ситуация, случай в...","[мчс россия сахалинский область, россия сахали..."
Тема 3:,"[соблюдать скоростной режим, правило дорожный ...","[рекомендовать воздержаться поездка, гибдд при...","[быть внимательный осторожный, правило дорожны...","[быть внимательный осторожный, соблюдать мера ...","[чрезвычайный ситуация необходимый, ситуация н...","[чрезвычайный ситуация необходимый, ситуация н...","[убрать хозяйственный вещь двор балкон, убрать...","[главный управление мчс россия, управление мчс..."
Тема 4:,"[ребенок как действовать, ребенок как действов...","[как действовать время, как действовать время ...","[возникновение чрезвычайный ситуация, чрезвыча...","[ограничить выход здание, подземный переход по...","[главный управление мчс, управление мчс россия...","[главный управление мчс, управление мчс россия...","[управление мчс россия, главный управление мчс...","[горно лесной местность, выход горно лесной, в..."
Тема 5:,"[главный управление мчс, управление мчс россия...","[мчс россия смоленский, мчс россия смоленский ...","[мчс россия республика, подземный переход подъ...","[возникновение чрезвычайный ситуация, рекоменд...","[мчс россия республика калмыкия, россия респуб...","[единый номер вызов, номер вызов экстренный, е...","[необходимый мера обеспечение, весь необходимы...","[предел населенный пункт, держаться далекий ле..."
Тема 6:,"[возникновение чрезвычайный ситуация, чрезвыча...","[режим повысить готовность, перевести режим по...","[управление мчс россия, главный управление мчс...","[случай чрезвычайный ситуация, ситуация необхо...","[режим повысить готовность, соблюдать мера без...","[соблюдать скоростной режим, конструкция линия...","[население порывистый ветер, рекомендация насе...","[предел населенный пункт, предел населить пунк..."
Тема 7:,"[соблюдение требование безопасность, соблюдени...","[правило дорожный движение, соблюдать правило ...","[дело го чс, рекламный щит дерево, парковать м...","[дежурно диспетчерский служба, единый дежурно ...","[вызов экстренный служба, единый номер вызов, ...","[правило техника безопасность, рекомендовать ж...","[режим повысить готовность, быть внимательный ...","[погодный условие рекомендоваться, сложный пог..."
Тема 8:,"[башкортостан чрезвычайный ситуация, башкортос...","[желтый уровень опасность гололедица, желтый у...","[республика башкортостан чрезвычайный, башкорт...","[главный управление мчс, управление мчс россия...","[покинуть опасный место, отдых водоем гора, пр...","[ухудшение погодный условие, безопасность ребе...","[предел населить пункт, поездка предел населит...","[работать круглосуточный режим, барнаульский с..."


### Сделаем LDA посубъектам

In [36]:
wknown = wind_work[wind_work['Класс']>4]

In [105]:
import re
from nltk.tokenize import word_tokenize

def normalise_text(text):
    text = text.lower()

    # сурово регулярками выкидываем мусорные символы
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    text = text.replace('марий эл', 'марий-эл')
    text = text.replace('северная осетия', 'северная_осетия')
    text = text.replace('саха якутия', 'саха')
    text = text.replace('якутия', 'саха')
    text = text.replace('санктпетербург', 'санкт-петербург')
    text = text.replace('санктпетеpбуpг', 'санкт-петербург')
    text = text.replace('забаикальскии краи', 'забайкальский')
    text = text.replace('ленингpадская', 'ленинградская')
    # Убираем стоп-слова
    words = text.split()
    filtered_words = [word for word in words if word not in stops]
    
    return ' '.join(filtered_words).strip()

def to_loc_num(loc):
    lst = []
    loc = loc.split()
    for i in loc:
        lst.append(subs_nums[i])
    return lst
    
stops = ['округ', 'область', 'автономный', 'автономная', 'г', 'край', 'республика']
wind_work['Location_ATD4'] = wind_work['Location_ATD4'].fillna('')
wind_work['Location_clean'] = wind_work['Location_ATD4'].apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(wind_work['Location_clean'].values)))

subs = {
    "#PAD#": 0, "#UNK#": 1
}

for word, _ in word_cnt.most_common():
    if word not in stops:
        subs[word] = _
        
subs_nums = {key: index for index, key in enumerate(list(subs.keys())[2:])}
wind_work['loc_num'] = wind_work['Location_clean'].apply(to_loc_num)

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\3874988716.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_work['Location_ATD4'] = wind_work['Location_ATD4'].fillna('')
C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\3874988716.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_work['Location_clean'] = wind_work['Location_ATD4'].apply(normalise_text)
C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\3874988716.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [106]:
def to_def_sub(value):

    if str(value).isdigit():
        return list(subs_nums.keys())[int(value)]
    else:
        return ''
        
wknown = wind_work.explode('loc_num')
wknown['Location_clean'] = wknown['loc_num'].apply(to_def_sub)

In [107]:
wknown['Location_clean'].value_counts()

Location_clean
краснодарский      1262
астраханская        725
приморский          716
московская          663
калмыкия            572
                   ... 
севастополь          14
северная_осетия      13
пензенская           13
чеченская             7
магаданская           2
Name: count, Length: 86, dtype: int64

In [108]:
wknown[['Класс', 'loc_num']].corr()

,Класс,loc_num
Класс,1.000000,-0.012076
loc_num,-0.012076,1.000000


### LDA всего текста по субъектам

In [173]:
wind_dict_all = {}
for i in subs_nums:
    try:
        themes = to_lda(wknown[wknown['Location_clean'] == i], ngram_range=(3, 5), n_themes=1, n_words=10)
        if themes:
            wind_dict_all[i] = themes
    except Exception as e:
        # Вы можете добавить вывод сообщения об ошибке, если нужно
        print(f"Ошибка при обработке {i}: {e}")


In [151]:
# # Можно настроить гиперпараметры отдельно для каждого субъекта

# wind_dict_krasnodar = to_lda(wknown[wknown['Location_clean']=='краснодарский'],
#                                    ngram_range=(3, 5), n_themes =1, n_words = 15)
# wind_dict_krasnodar

In [182]:
wind_all_df = pd.DataFrame(wind_dict_all).T
wind_all_df

,5,6,7,8,9,10,11,12
краснодарский,"{'Тема 1:': ['широко раскинуть рука', 'день те...","{'Тема 1:': ['г тимашевск ул', 'шифер г тимаше...","{'Тема 1:': ['слабо укрепить конструкция', 'во...","{'Тема 1:': ['шквалистый усиление ветер', 'сил...","{'Тема 1:': ['шквалистый усиление ветер', 'гра...","{'Тема 1:': ['место край ожидаться', 'ливень с...","{'Тема 1:': ['риа новость виталий', 'новость в...","{'Тема 1:': ['северо восточный ветер', 'очень ..."
астраханская,{'Тема 1:': ['территория астраханский область'...,{'Тема 1:': ['январь прогнозироваться гололедн...,"{'Тема 1:': ['астрахань ул волжский', 'г астра...","{'Тема 1:': ['адрес г астрахань ул волжский', ...","{'Тема 1:': ['г астрахань ул волжский', 'г аст...","{'Тема 1:': ['данные астраханский цгмс', 'г ас...",{'Тема 1:': ['резервный источник электроснабже...,{'Тема 1:': ['аварийно спасательный имущество'...
приморский,{},"{'Тема 1:': ['уссурийский городской округ', 'д...","{'Тема 1:': ['количество ветровый нагрузка', '...","{'Тема 1:': ['подъем уровень вода', 'более час...","{'Тема 1:': ['очень сильный дождь', 'более час...","{'Тема 1:': ['гололедица снежный накат', 'вете...","{'Тема 1:': ['подъем уровень вода', 'очень сил...","{'Тема 1:': ['залив петр великий', 'очень силь..."
московская,"{'Тема 1:': ['рабочий личный состав', 'ед кдм ...","{'Тема 1:': ['вечер пятница перейти', 'вечер п...","{'Тема 1:': ['московский область ожидаться', '...","{'Тема 1:': ['усиление ветер порыв', 'московск...","{'Тема 1:': ['ухудшение погодный условие', 'ус...","{'Тема 1:': ['территория московский область', ...","{'Тема 1:': ['режим повысить готовность', 'тра...",{}
калмыкия,{},{},"{'Тема 1:': ['мчс россия республика', 'мчс рос...","{'Тема 1:': ['россия республика калмыкия', 'мч...","{'Тема 1:': ['россия республика калмыкия', 'мч...","{'Тема 1:': ['республика калмыкия г', 'республ...","{'Тема 1:': ['и ленин д', 'республика калмыкия...",{}
...,...,...,...,...,...,...,...,...
севастополь,{},{},{},{},"{'Тема 1:': ['температура воздух ночь', 'гидро...",{'Тема 1:': ['штормовой предупреждение завтра ...,{},"{'Тема 1:': ['риа новость дмитрий', 'риа новос..."
пензенская,{},{},{},{'Тема 1:': ['пензенский цгмс филиал фгб приво...,{},{},{},{}
северная_осетия,{},{},{},{},{},{'Тема 1:': ['это река возможный подъем уровен...,"{'Тема 1:': ['бригада филиал незамедлительно',...",{}
чеченская,{},{},{},{},{},"{'Тема 1:': ['антенна выключить телевизор', 'я...",{},{}


### LDA по описаниям ОПЯ по субъектам

In [134]:
event['Location_ATD4'] = event['Location_ATD4'].fillna('')
event['Location_clean'] = event['Location_ATD4'].apply(normalise_text)
event['loc_num'] = event['Location_clean'].apply(to_loc_num)
event = event.explode('loc_num')
event['Location_clean'] = event['loc_num'].apply(to_def_sub)

In [170]:
event_dict_all = {}
for i in subs_nums:
    try:
        themes = to_lda(event[event['Location_clean'] == i], ngram_range=(2, 5), n_themes=1, n_words=10)
        if themes:
            event_dict_all[i] = themes
    except Exception as e:
        # Вы можете добавить вывод сообщения об ошибке, если нужно
        print(f"Ошибка при обработке {i}: {e}")


Ошибка при обработке дагестан: empty vocabulary; perhaps the documents only contain stop words
Ошибка при обработке ивановская: empty vocabulary; perhaps the documents only contain stop words


In [155]:
# # Можно настроить гиперпараметры отдельно для каждого субъекта
# event_dict_krasnodar = to_lda(event[event['Location_clean']=='краснодарский'],
#                                    ngram_range=(2, 5), n_themes = 1, n_words = 15)
# event_dict_krasnodar

In [181]:
event_all_df = pd.DataFrame(event_dict_all).T
event_all_df

,5,6,7,8,9,10,11,12
краснодарский,"{'Тема 1:': ['толщина лед', 'уменьшение толщин...",{'Тема 1:': ['электропередача нарушение работа...,"{'Тема 1:': ['усиление ветер', 'сильный дождь'...","{'Тема 1:': ['сильный дождь', 'усиление ветер'...","{'Тема 1:': ['сильный дождь', 'усиление ветер'...","{'Тема 1:': ['сильный дождь', 'усиление ветер'...","{'Тема 1:': ['сильный дождь', 'сильный ветер',...","{'Тема 1:': ['сильный ветер', 'ветер порыв', '..."
астраханская,{'Тема 1:': ['чрезвычайный пожароопасность зап...,{},"{'Тема 1:': ['северо восточный', 'температура ...","{'Тема 1:': ['усиление ветер', 'ветер порыв', ...","{'Тема 1:': ['сильный дождь', 'ожидаться сильн...","{'Тема 1:': ['сильный дождь', 'дождь ливень', ...","{'Тема 1:': ['шквалистый ветер достигать', 'шк...",{'Тема 1:': ['усиление северо западный западны...
приморский,{},"{'Тема 1:': ['снежный накат', 'температура воз...","{'Тема 1:': ['дождь снег', 'снег количество', ...","{'Тема 1:': ['сильный дождь', 'усиление ветер'...","{'Тема 1:': ['сильный дождь', 'дождь количеств...","{'Тема 1:': ['очень сильный', 'сильный осадок'...","{'Тема 1:': ['очень сильный', 'сильный дождь',...","{'Тема 1:': ['сильный дождь', 'очень сильный',..."
московская,"{'Тема 1:': ['температура воздух', 'максимальн...","{'Тема 1:': ['снегопад который', 'потепление в...","{'Тема 1:': ['ветер порыв', 'усиление ветер', ...","{'Тема 1:': ['ветер порыв', 'усиление ветер', ...","{'Тема 1:': ['ветер порыв', 'мокрый снег', 'ус...","{'Тема 1:': ['ветер порыв', 'усиление ветер', ...",{'Тема 1:': ['сек заморозка минус градус образ...,{}
калмыкия,{},{},{'Тема 1:': ['гололедно изморозевый отложение'...,"{'Тема 1:': ['усиление ветер', 'ветер порыв', ...","{'Тема 1:': ['усиление ветер', 'сильный дождь'...","{'Тема 1:': ['усиление ветер', 'сильный дождь'...","{'Тема 1:': ['очень сильный', 'западный ветер'...",{}
...,...,...,...,...,...,...,...,...
севастополь,{},{},{},{},"{'Тема 1:': ['день февраль', 'ветер южный', 'ю...","{'Тема 1:': ['сильный дождь ветер порыв', 'сил...",{},"{'Тема 1:': ['штормовой погода ливень', 'урага..."
пензенская,{},{},{},"{'Тема 1:': ['ожидаться сильный ливень', 'силь...",{},{},{},{}
северная_осетия,{},{},{},{},{},"{'Тема 1:': ['шквалистый усиление', 'шквалисты...",{'Тема 1:': ['скорость ветер']},{}
чеченская,{},{},{},{},{},{'Тема 1:': ['шквалистый ветер река возможный ...,{},{}


### LDA по ущербам от ОПЯ по субъектам

In [139]:
damage['Location_ATD4'] = damage['Location_ATD4'].fillna('')
damage['Location_clean'] = damage['Location_ATD4'].apply(normalise_text)
damage['loc_num'] = damage['Location_clean'].apply(to_loc_num)
damage = damage.explode('loc_num')
damage['Location_clean'] = damage['loc_num'].apply(to_def_sub)

In [174]:
damage_dict_all = {}
for i in subs_nums:
    try:
        themes = to_lda(damage[damage['Location_clean'] == i], ngram_range=(2, 5), n_themes=1, n_words=10)
        if themes:
            damage_dict_all[i] = themes
    except Exception as e:
        # Вы можете добавить вывод сообщения об ошибке, если нужно
        print(f"Ошибка при обработке {i}: {e}")


In [148]:
# # Можно настроить гиперпараметры отдельно для каждого субъекта
# damage_dict_krasnodar = to_lda(damage[damage['Location_clean']=='краснодарский'],
#                                    ngram_range=(2, 5), n_themes = 1, n_words = 15)
# damage_dict_krasnodar

In [180]:
damage_all_df = pd.DataFrame(damage_dict_all).T
damage_all_df

,5,6,7,8,9,10,11,12
краснодарский,{'Тема 1:': ['частичный обрушение кровля мкд у...,"{'Тема 1:': ['г тимашевск ул', 'тимашевск ул',...","{'Тема 1:': ['электропередача нарушение', 'свя...","{'Тема 1:': ['линия связь электропередача', 'л...","{'Тема 1:': ['падение дерево', 'линия связь', ...","{'Тема 1:': ['коммунальный служба', 'укрепить ...","{'Тема 1:': ['повалить дерево', 'падение дерев...","{'Тема 1:': ['линия электропередача', 'ветер в..."
астраханская,{},{},"{'Тема 1:': ['существовать вероятность', 'веро...","{'Тема 1:': ['вероятность возникновение', 'сущ...","{'Тема 1:': ['обрыв линия', 'существовать веро...","{'Тема 1:': ['обрыв линия', 'линия электропере...","{'Тема 1:': ['учебный процесс', 'пройти нескол...",{'Тема 1:': ['электропередача связь прибрежный...
приморский,{},"{'Тема 1:': ['быть осторожный', 'круглосуточны...",{'Тема 1:': ['явление налипание мокрый снег до...,"{'Тема 1:': ['участок дорога', 'хозяйственный ...","{'Тема 1:': ['снежный накат', 'гололедица снеж...","{'Тема 1:': ['снежный накат', 'гололедица снеж...","{'Тема 1:': ['снежный накат', 'гололедный явле...","{'Тема 1:': ['подтопление территория', 'выход ..."
московская,"{'Тема 1:': ['противогололедный материал', 'бы...","{'Тема 1:': ['дорога возможный образование', '...","{'Тема 1:': ['снежный занос', 'возможный голол...","{'Тема 1:': ['снежный занос', 'занос гололедиц...","{'Тема 1:': ['возможный образование', 'видимос...","{'Тема 1:': ['повалить дерево', 'городской окр...",{'Тема 1:': ['ураган принести много разрушение...,{}
калмыкия,{},{},"{'Тема 1:': ['жизнеобеспечение население', 'ве...","{'Тема 1:': ['происшествие связать', 'поврежде...","{'Тема 1:': ['повреждение кровля', 'происшеств...","{'Тема 1:': ['происшествие связать', 'поврежде...","{'Тема 1:': ['происшествие связать', 'поврежде...",{}
...,...,...,...,...,...,...,...,...
севастополь,{},{},{},{},{'Тема 1:': ['электропередача падение дерево р...,{},{},"{'Тема 1:': ['повалить дерево', 'повалить дере..."
пензенская,{},{},{},{'Тема 1:': ['электроснабжение потребитель нес...,{},{},{},{}
северная_осетия,{},{},{},{},{},{},{'Тема 1:': ['энергетика восстанавливать элект...,{}
чеченская,{},{},{},{},{},{'Тема 1:': ['электроэнергия специалист рекоме...,{},{}


### LDA по мерам и мероприятиям по субъектам

In [109]:
activities['Location_ATD4'] = activities['Location_ATD4'].fillna('')
activities['Location_clean'] = activities['Location_ATD4'].apply(normalise_text)
activities['loc_num'] = activities['Location_clean'].apply(to_loc_num)
activities = activities.explode('loc_num')
activities['Location_clean'] = activities['loc_num'].apply(to_def_sub)

In [175]:
activities_dict_all = {}
for i in subs_nums:
    try:
        themes = to_lda(activities[activities['Location_clean'] == i], ngram_range=(2, 5), n_themes=1, n_words=10)
        if themes:
            activities_dict_all[i] = themes
    except Exception as e:
        # Вы можете добавить вывод сообщения об ошибке, если нужно
        print(f"Ошибка при обработке {i}: {e}")


In [159]:
# # Можно настроить гиперпараметры отдельно для каждого субъекта
# activities_dict_krasnodar = to_lda(activities[activities['Location_clean']=='краснодарский'],
#                                    ngram_range=(2, 5), n_themes = 1, n_words = 15)
# activities_dict_krasnodar

In [179]:
activities_all_df = pd.DataFrame(activities_dict_all).T
activities_all_df

,5,6,7,8,9,10,11,12
краснодарский,{'Тема 1:': ['энергоснабжение установить решен...,"{'Тема 1:': ['житель гость', 'быть осторожный'...","{'Тема 1:': ['шаткий строение', 'щит дерево', ...","{'Тема 1:': ['чрезвычайный ситуация', 'случай ...","{'Тема 1:': ['чрезвычайный ситуация', 'ситуаци...","{'Тема 1:': ['чрезвычайный ситуация', 'быть вн...","{'Тема 1:': ['чрезвычайный ситуация', 'необход...","{'Тема 1:': ['чрезвычайный ситуация', 'принять..."
астраханская,{'Тема 1:': ['чрезвычайный ситуация звонить те...,{'Тема 1:': ['случай непредвиденный ситуация о...,"{'Тема 1:': ['главный управление', 'главный уп...","{'Тема 1:': ['экстренный ситуация', 'обращатьс...","{'Тема 1:': ['главный управление', 'управление...","{'Тема 1:': ['главный управление', 'мера преду...","{'Тема 1:': ['резервный источник', 'аварийный ...",{'Тема 1:': ['швартов суд стоить причал провер...
приморский,{},"{'Тема 1:': ['быть задействовать', 'расчистка ...","{'Тема 1:': ['спасатель мчс', 'спасатель мчс р...","{'Тема 1:': ['поездка предел', 'житель гость',...","{'Тема 1:': ['повысить готовность', 'воздержат...","{'Тема 1:': ['поездка предел', 'повысить готов...","{'Тема 1:': ['поездка предел', 'предел населит...","{'Тема 1:': ['воздержаться поездка', 'рекоменд..."
московская,"{'Тема 1:': ['соблюдать пдд', 'служба продолжа...","{'Тема 1:': ['скоростной режим', 'дорожный слу...","{'Тема 1:': ['быть внимательный', 'скоростной ...","{'Тема 1:': ['быть внимательный', 'шаткий конс...","{'Тема 1:': ['дорожный служба', 'поездка личны...","{'Тема 1:': ['быть внимательный', 'мчс россия'...",{'Тема 1:': ['спецтехника включая резервный ис...,{}
калмыкия,{},{},"{'Тема 1:': ['мчс россия', 'управление мчс', '...","{'Тема 1:': ['мчс россия', 'россия республика'...","{'Тема 1:': ['главный управление мчс', 'управл...","{'Тема 1:': ['мчс россия', 'главный управление...","{'Тема 1:': ['балкон дом', 'вещь двор', 'хозяй...",{}
...,...,...,...,...,...,...,...,...
севастополь,{},{},{},{},"{'Тема 1:': ['ребенок пожилой', 'правило безоп...",{'Тема 1:': ['усилить режим работа сотрудник в...,{},"{'Тема 1:': ['выходной день', 'оперативный шта..."
пензенская,{},{},{},"{'Тема 1:': ['сотрудник мчс', 'коммунальный сл...",{},{},{},{}
северная_осетия,{},{},{},{},{},"{'Тема 1:': ['быть внимательный осторожный', '...",{'Тема 1:': ['энергетика продолжать работа сег...,{}
чеченская,{},{},{},{},{},"{'Тема 1:': ['чрезвычайный происшествие', 'тер...",{},{}


In [187]:
# # Сохраняемся

# with open('files//models//LDA//wind_dict_all.pkl', 'wb') as file:
#     pickle.dump(wind_dict_all, file)

# with open('files//models//LDA//event_dict_all.pkl', 'wb') as file:
#     pickle.dump(event_dict_all, file)

# with open('files//models//LDA//damage_dict_all.pkl', 'wb') as file:
#     pickle.dump(damage_dict_all, file)

# with open('files//models//LDA//activities_dict_all.pkl', 'wb') as file:
#     pickle.dump(activities_dict_all, file)

In [188]:
# with open('files//models//LDA//wind_dict.pkl', 'wb') as file:
#     pickle.dump(wind_dict, file)

# with open('files//models//LDA//event_dict.pkl', 'wb') as file:
#     pickle.dump(event_dict, file)

# with open('files//models//LDA//damage_dict.pkl', 'wb') as file:
#     pickle.dump(damage_dict, file)

# with open('files//models//LDA//activities_dict.pkl', 'wb') as file:
#     pickle.dump(activities_dict, file)

In [184]:
# wind_all_df.to_excel('files//models//LDA//wind_all_df.xlsx', index = False)
# event_all_df.to_excel('files//models//LDA//event_all_df.xlsx', index = False)
# damage_all_df.to_excel('files//models//LDA//damage_all_df.xlsx', index = False)
# activities_all_df.to_excel('files//models//LDA//activities_all_df.xlsx', index = False)

In [194]:
# wind_dict_df.to_excel('files//models//LDA//wind_dict_df.xlsx', index = False)
# event_dict_df.to_excel('files//models//LDA//event_dict_df.xlsx', index = False)
# damage_dict_df.to_excel('files//models//LDA//damage_dict_df.xlsx', index = False)
# activities_dict_df.to_excel('files//models//LDA//activities_dict_df.xlsx', index = False)

## Обучение Random Forest на задачу классификации новостей о сильных ветрах на классы интенсивности по Бофорту

In [95]:
wind_pred = wind_work.copy().dropna(subset = 'Text')
data = wind_pred[wind_pred['Класс'] > 4]
data['Класс'].value_counts()

Класс
8     2582
10    1913
9     1826
7     1036
11     447
12     281
6      181
5      152
Name: count, dtype: int64

### Классы не сбалансированы по количеству, объединим их группы, будем предсказывать не 8 классов, а 5

In [96]:
def to_change_classes(value):
    if value<=7:
        return 0
    elif value == 8:
        return 1
    elif value == 9:
        return 2
    elif value == 10:
        return 3
    else:
        return 4

data['Класс_группа'] = data['Класс'].apply(to_change_classes)

C:\Users\Mariia\AppData\Local\Temp\ipykernel_1936\2660667053.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Класс_группа'] = data['Класс'].apply(to_change_classes)


In [97]:
data['Класс_группа'].value_counts()

Класс_группа
1    2582
3    1913
2    1826
0    1369
4     728
Name: count, dtype: int64

### Уберем из текство новостей всю информацию о скорости ветра и другие числа

In [98]:
start_time = time.time()

# Предобработка текста
X = data['Text']
y = data['Класс_группа']

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

custom_stop_words = set(['м', 'с', 'мм', 'c', 'км', 'метр',
                     'секунда', 'лицензия', 'creative', 'commons', 'attribution'])
stops_ru = set(stopwords.words('russian'))
stops = list(custom_stop_words|stops_ru)

# Векторизация
vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Zа-яА-ЯёЁ]+\b', 
                             stop_words=stops, 
                             ngram_range=(1, 2),
                             max_df=0.85,
                             min_df=40)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)



# Обучение модели RandomForest с учетом весов классов
model = RandomForestClassifier(random_state=42, n_estimators = 350)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# # Прогнозирование для оставшихся новостей
# X_test = wind_pred[wind_pred['Класс'] < 5]['Text']
# X_test_vec = vectorizer.transform(X_test)
# predictions = model.predict(X_test_vec)

# # Сохранение результатов
# wind_pred.loc[wind_pred['Класс'] < 5, 'Класс'] = predictions


end_time = time.time()

execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")

              precision    recall  f1-score   support

           0       0.71      0.66      0.68       257
           1       0.54      0.73      0.62       524
           2       0.59      0.44      0.50       343
           3       0.71      0.65      0.68       402
           4       0.82      0.59      0.68       158

    accuracy                           0.63      1684
   macro avg       0.67      0.61      0.63      1684
weighted avg       0.64      0.63      0.63      1684


Время выполнения: 1.00 минут


### Попробуем НЕ убирать скорости ветра и другие числа

#### `Random Forest`. классификация на 5 классов

In [82]:
start_time = time.time()


# Предобработка текста
X = data['Text']
y = data['Класс_группа']

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


stops_ru = list(stopwords.words('russian'))


vectorizer = CountVectorizer(
    stop_words=stops_ru,
    ngram_range=(1, 2),  # Изменение диапазона n-грамм
    max_df=0.85,          # Игнорировать слова, которые встречаются более чем в 85% новостей
    min_df=40             # Игнорировать слова, которые встречаются менее чем в 30 новостях
)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

model = RandomForestClassifier(random_state=42,
                               n_estimators = 350,
                               n_jobs = -1)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# # Прогнозирование для оставшихся новостей
# X_test = wind_pred[wind_pred['Класс'] < 5]['Text']
# X_test_vec = vectorizer.transform(X_test)
# predictions = model.predict(X_test_vec)

# # Сохранение результатов
# wind_pred.loc[wind_pred['Класс'] < 5, 'Класс'] = predictions

end_time = time.time()

execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")

              precision    recall  f1-score   support

           0       0.86      0.91      0.88       257
           1       0.88      0.91      0.89       524
           2       0.83      0.86      0.84       343
           3       0.90      0.88      0.89       402
           4       0.95      0.73      0.83       158

    accuracy                           0.88      1684
   macro avg       0.88      0.86      0.87      1684
weighted avg       0.88      0.88      0.87      1684


Время выполнения: 0.00 минут


#### `Random Forest`. классификация на все 8 классов

In [87]:
start_time = time.time()

# Предобработка текста
X = data['Text']
y = data['Класс']

# Разделение данных
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


stops_ru = list(stopwords.words('russian'))


vectorizer = CountVectorizer(
    stop_words=stops_ru,
    ngram_range=(1, 2),  # Изменение диапазона n-грамм
    max_df=0.85,          # Игнорировать слова, которые встречаются более чем в 85% новостей
    min_df=40             # Игнорировать слова, которые встречаются менее чем в 30 новостях
)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)


model = RandomForestClassifier(random_state=42,
                               n_estimators = 350,
                               n_jobs = -1)
model.fit(X_train_vec, y_train)

# Оценка модели
y_pred = model.predict(X_val_vec)
print(classification_report(y_val, y_pred))

# # Прогнозирование для оставшихся новостей
# X_test = wind_pred[wind_pred['Класс'] < 5]['Text']
# X_test_vec = vectorizer.transform(X_test)
# predictions = model.predict(X_test_vec)

# # Сохранение результатов
# wind_pred.loc[wind_pred['Класс'] < 5, 'Класс'] = predictions

end_time = time.time()

execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time//60:.2f} минут")

              precision    recall  f1-score   support

           5       0.89      0.59      0.71        29
           6       0.88      0.41      0.56        34
           7       0.85      0.85      0.85       194
           8       0.85      0.91      0.88       524
           9       0.82      0.88      0.85       343
          10       0.85      0.90      0.87       402
          11       0.93      0.65      0.76        97
          12       0.97      0.52      0.68        61

    accuracy                           0.85      1684
   macro avg       0.88      0.71      0.77      1684
weighted avg       0.85      0.85      0.84      1684


Время выполнения: 0.00 минут
